In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
import nltk
import numpy as np
# import pyLDAvis
import pyLDAvis.gensim_models
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [45]:
data = pd.read_csv('csvfile_global_twitter_processed.csv')

In [46]:
data.head()

,created_at,id,id_str,original_text,truncated,display_text_range,entities,metadata,source,user,...,favorite_count,favorited,retweeted,lang,polarity,subjectivity,LENGTH,preprocessed,sentiment,clean
0,2022-08-07 22:31:20+00:00,1556407673284861952,1556407673284861952,RT @i_ameztoy: Extra random image (I):\n\nLets...,False,"[0, 143]","{'hashtags': [{'text': 'City', 'indices': [132...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...","{'id': 3418339671, 'id_str': '3418339671', 'na...",...,0,False,False,en,-1.250000e-01,0.190625,23,rt iameztoy extra random imag let focu one spe...,negative,rt iameztoy extra random imag let focu one spe...
1,2022-08-07 22:31:16+00:00,1556407654373027840,1556407654373027840,RT @IndoPac_Info: #China's media explains the ...,False,"[0, 140]","{'hashtags': [{'text': 'China', 'indices': [18...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...","{'id': 1003562923, 'id_str': '1003562923', 'na...",...,0,False,False,en,-1.000000e-01,0.100000,24,rt indopacinfo china medium explain militari r...,negative,rt indopacinfo china medium explain militari r...
2,2022-08-07 22:31:07+00:00,1556407616208773122,1556407616208773120,"China even cut off communication, they don't a...",False,"[0, 161]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...","{'id': 1132320481506406400, 'id_str': '1132320...",...,0,False,False,en,0.000000e+00,0.000000,26,china even cut commun dont anwer phonecal u cl...,neutral,china even cut commun dont anwer phonecal u cl...
3,2022-08-07 22:31:06+00:00,1556407614602461185,1556407614602461184,"Putin to #XiJinping : I told you my friend, Ta...",False,"[0, 179]","{'hashtags': [{'text': 'XiJinping', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...","{'id': 1132320481506406400, 'id_str': '1132320...",...,0,False,False,en,1.000000e-01,0.350000,33,putin xijinp tell friend taiwan vassal state i...,positive,putin xijinp tell friend taiwan vassal state i...
4,2022-08-07 22:31:04+00:00,1556407604238245889,1556407604238245888,"RT @ChinaUncensored: I’m sorry, I thought Taiw...",False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...","{'id': 1260798065373523969, 'id_str': '1260798...",...,0,False,False,en,-6.938894e-18,0.556250,21,rt chinauncensor i’m sorri think taiwan indepe...,negative,rt chinauncensor i’m sorri think taiwan indepe...


In [47]:
wnl = WordNetLemmatizer()

def removePunc(myWord):
    """Function to remove punctuation from string inputs"""
    if myWord is None:
        return myWord
    else:
        return sub('[.:;()/!&-*@$,?^\d+]','',myWord)
        
def removeAscii(myWord):
    """Function to remove ascii from string input"""
    if myWord is None:
        return myWord
    else:
        return str(sub(r'[^\x00-\x7F]+','', myWord.strip()))

def lemmatize(myWord):
    """Function to lemmatize words"""
    if myWord is None:
        return myWord
    else:
        return str(wnl.lemmatize(myWord))

def removeStopWords(myWord):
    """Function to remove stop words"""
    if myWord is None:
        return myWord
    if myWord not in str(stopwords.words('english')):
        return myWord

def removeLinkUser(myWord):
    """Function to remove web addresses and twitter handles"""
    if not myWord.startswith('@') and not myWord.startswith('http'):
        return myWord

def prepText(myWord):
    """Final text pre-processing function"""
    return removeStopWords(
        lemmatize(
            removeAscii(
                removePunc(
                    removeLinkUser(
                        myWord.lower()
                    )
                )
            )
        )
    )

<>:8: DeprecationWarning: invalid escape sequence \d
<>:8: DeprecationWarning: invalid escape sequence \d
<>:8: DeprecationWarning: invalid escape sequence \d
C:\Users\hp\AppData\Local\Temp/ipykernel_28336/3133919804.py:8: DeprecationWarning: invalid escape sequence \d
  return sub('[.:;()/!&-*@$,?^\d+]','',myWord)


In [48]:
def filterTweetList(tweetList):
    """Remove stop words, lemmatize, and clean all tweets"""
    return [[prepText(word) for word
                in tweet.split()
                    if prepText(word) is not None]
                for tweet in tweetList]

In [49]:
cleanKagTweetList = filterTweetList(data['clean'])

In [50]:
cleanKagTweetList[:5]

[['iameztoy',
  'extra',
  'random',
  'imag',
  'let',
  'focu',
  'one',
  'specif',
  'zone',
  'western',
  'coast',
  'longj',
  'district',
  'taichung',
  'citi',
  'ta'],
 ['indopacinfo',
  'china',
  'medium',
  'explain',
  'militari',
  'reason',
  'area',
  'drill',
  'taiwan',
  'strait',
  'read',
  'label',
  'pi'],
 ['china',
  'even',
  'cut',
  'commun',
  'dont',
  'anwer',
  'phonecal',
  'clown',
  'zelenskyyua',
  'enter',
  'stage',
  'ask',
  'xijinp',
  'chang',
  'putin',
  'mind'],
 ['putin',
  'xijinp',
  'tell',
  'friend',
  'taiwan',
  'vassal',
  'state',
  'includ',
  'nuke',
  'much',
  'like',
  'ukrainian',
  'model',
  'warn',
  'take',
  'pelosi',
  'open',
  'china',
  'eye'],
 ['chinauncensor',
  'sorri',
  'think',
  'taiwan',
  'independ',
  'countri',
  'govern',
  'currenc',
  'militari',
  'travel']]

In [51]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=50,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [53]:
"""Create model objects"""
kagDict   = makeDict(cleanKagTweetList)
kagCorpus = makeCorpus(cleanKagTweetList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'LDAmodel')
corpora.MmCorpus.serialize('Corpus.mm', kagCorpus)
kagDict.save('Dictionary.dict')

kagLda = SaveLoad.load('LDAmodel')
kagDict = corpora.Dictionary.load('Dictionary.dict')
kagCorpus = corpora.MmCorpus('Corpus.mm')

In [54]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

C:\Users\hp\anaconda3\envs\Excell-Chatbot\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [55]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [56]:
newIdx = translateLdaIdx(kagLda,ldaViz)

def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [ ]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)

def sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList)
    srtIdx = list(mySortedDf[mySortedDf[myTopic]>myTopicThresh].index)
    return srtIdx

def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [ ]:
sortedIdx = sortByTopicToIdx(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.1)

In [ ]:
# Word cloud for topic 
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [ ]:
# It can be seen that most tweeted words are related to war in unkraine and economic crisis.
makeWordCloud(cleanKagTweetList,kagDf,'topic2',myTopicThresh=0.2)